In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from PIL import Image as PIL_Image
from PIL import ImageOps as PIL_ImageOps
import time
from collections import Counter
import re
import seaborn as sns
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
data_path = '/path/to/pew_test.json'
images_data = '/path/to/pew_test_imgs'

In [ ]:
def json2pd(file_path):
    with open(file_path) as f:
        temp_data = [json.loads(line) for line in f]
    df = pd.DataFrame(temp_data)
    return df

In [ ]:
images_list = os.listdir(images_data)
images = [images_data + "/" + i for i in images_list]

In [ ]:
image_i_to_dict = {}
for i in range(len(images)):
    image_i_to_dict[images_list[i].split(".")[0]] = images[i]

In [ ]:
test_df = json2pd(data_path)
test_df.head()

In [ ]:
# Utility Function
def generate_input_message(system_prompt, user_prompt):
  messages = []
  messages.append({"role": "system", "content": system_prompt})
  messages.append({"role": "user", "content": user_prompt})

  return messages

### **SYSTEM and USER Prompts**

### **Reflection**

In [ ]:
############## REFLECTION PROMPTS#########################
def initialize_reflection_prompt():
  ## REFLECTION
  # ======================================== $$Initial Reflection$$ ========================================================
  initial_system_reflection_prompt = f'As an intelligent data analyst and insight extraction specialist, your role is to generate a ‘reflection’ from data tables that must cover every important detail that can be observed in the data tables. Pay attention to small details and nuances as well as any trends or outliers in the given tables.'
  initial_user_reflection_prompt = f'''### Task Description:
  Given the data tables corresponding to a data story in the input, your task is the following:
  1. Generate a coherent ‘reflection’ on the data tables given in the input, in bullet points. Here, ‘reflection’ is defined as the systematic examination and interpretation of data tables to narrate a coherent story, involving a comprehensive understanding of the data structure, identification of key variables, analysis of data distribution and trends, and understanding of the data’s broader context.
  2. Identify and discuss the most impactful insights from the data tables. Focus on elements that significantly influence the narrative or findings, such as critical trends, notable patterns, and significant outliers.
  3. Determine the importance of details based on their relevance to the overall story, potential implications, and their statistical significance.
  4. Explain how different attributes of the data tables are interconnected. Highlight any causal relationships, correlations, or patterns that emerge from the data.
  5. Discuss any observed trends or outliers, explaining their potential implications or causes.

  ### Additional Guidelines:
  - The output must be in plain text and structured in bullet points.
  - Generate the response ‘reflection’ in between two <reflection> tags.
  '''

  return initial_system_reflection_prompt, initial_user_reflection_prompt


def reflection_verification_prompt():
  # ======================================== $$Reflection Verification$$ ======================================================
  system_reflection_verification_prompt = f'As an analytical critic, your role is to meticulously examine the alignment between data presented in tables and the narrative provided in a reflection. Focus on identifying any discrepancies in the details and the overall message conveyed. Consider not just the numbers but also the context and implications of the data.'
  user_reflection_verification_prompt = '''### Task Description:
  Given the data tables and a reflection corresponding to a data story in the input, your task is the following:
  1. Carefully analyze the data tables and the reflection. Identify any discrepancies or inconsistencies, focusing on numerical data, contextual interpretations, and the reflection’s fidelity to the data. Discrepancies might include but are not limited to incorrect data interpretation, or overlooked details.
  2. Prepare a plan to revise the reflection if needed, and output the revision plan. Otherwise just output: ‘No revision needed’.
  3. The revision plan if needed must coherently and logically relate to the attributes of the data.
  4. Be as specific as possible.

  ### Additional Guidelines:
  - The output must be in plain text and structured in bullet points.
  - Generate the response ‘reflection’ in between two <reflection> tags.
  '''

  return system_reflection_verification_prompt, user_reflection_verification_prompt

def reflection_revision_prompt():
  # ======================================== $$Reflection Revision$$ ==========================================================
  system_reflection_revision_prompt = f'As an intelligent data analyst and insight extraction specialist, your role is to generate a ‘reflection’ from data tables that must cover every important detail that can be observed in the data tables. Pay attention to small details and nuances as well as any trends or outliers in the given tables.'
  user_reflection_revision_prompt = f'''### Task Description:
  Given the data tables corresponding to a data story and a revision plan for reflection in the input, your task is the following:
  1. Revise the reflection according to the revision plan. Pay attention to small details and nuances and any trends or outliers in the given tables.
  2. The generated reflection must coherently and logically relate to the attributes of the data.
  3. Be as specific as possible.

  ### Additional Guidelines:
  - The output must be in plain text and structured in bullet points.
  - Generate the response ‘reflection’ in between two <reflection> tags.
  '''

  return system_reflection_revision_prompt, user_reflection_revision_prompt

### **Outline**

In [ ]:
def initialize_outline_prompt(intention):
  # ======================================== $$Initial Outline with Intent$$ ======================================================
  initial_system_outline_prompt = f'You are an expert at generating outlines for data stories. The generated outline should cover every important detail that can be observed in the data tables. Pay attention to small details and nuances as well as any trends or outliers in the given tables.'
  initial_user_outline_prompt = f'''### Task Description:
  Given a reflection and the data tables corresponding to a data story in the input, you have the following tasks:
  1. Generate an outline of the story following a linear narrative structure considering the reflection and the data presented in the tables. A linear narrative structure is defined as the narrative structure that contain a start, a middle, and an end. Think of it as setting the scene, unveiling the adventure, and wrapping up with a satisfying conclusion.
  2. Each of the points in the outlinebreak it down into smaller points that spotlight specific aspects of the data. This could include: significant figures or patterns, noteworthy exceptions or deviations, comparisons or changes over time. Add instructions for visualizations, i.e., charts, where necessary.
  3. The data story’s overarching theme should focus on "{intention}". Make sure this sentiment is consistent throughout the outline.
  4. Remember, the essence of a compelling data story is not just in the numbers but in how you tell the tale, so inclusion of visualization instruction is of utmost importance.
  5. Be specific, be clear, and most importantly, be engaging. The generated outline must coherently and logically relate to the attributes of the data. Be as specific as possible.

  ### Additional Guidelines:
  - The output must be in plain text and structured in bullet points.
  - Generate the response outline in between two <outline> tags.
  '''

  return initial_system_outline_prompt, initial_user_outline_prompt



def outline_verification_prompt(intention):
  # ======================================== $$Outline Verification$$ =========================================================
  system_outline_verification_prompt = f'You are an intelligent critic, whose job is to identify inconsistencies between data presented in data tables, and a reflection and an outline. Pay attention to small details and nuances as well as any trends or outliers in the given tables.'
  user_outline_verification_prompt = f'''### Task Description:
  Given the data tables, a reflection and an outline corresponding to a data story in the input, your task is the following:
  1. Identify inconsistencies between the data presented in the tables, the reflection and the outline.
  2. Make sure the revision plan is consistent with the intention or the main theme of the story: "{intention}"
  3. Prepare a plan to revise the outline if needed, and output the revision plan. Otherwise just output: ‘No revision needed’.
  4. The revision plan must coherently and logically relate to the attributes of the data. Be as specific as possible.

  ### Additional Guidelines:
  - The output must be in plain text and structured in bullet points.
  - Generate the response outline in between two <outline> tags.
  '''

  return system_outline_verification_prompt, user_outline_verification_prompt

def outline_revision_prompt(intention):
  # ======================================== $$Outline Revision$$ =============================================================
  system_outline_revision_prompt = f'You are an expert at generating outlines for data stories. The generated outline should cover every important detail that can be observed in the data tables. Pay attention to small details and nuances as well as any trends or outliers in the given tables.'
  user_outline_revision_prompt = f'''### Task Description:
  Given the data tables, the revision plan and the outline corresponding to a data story in the input, your task is the following:
  1. Apply the changes suggested in the revision plan to the existing outline.
  2. Ensure Theme Consistency: The data story’s overarching theme, defined as "{intention}", should be clearly reflected throughout the revised outline.
  3. Adjust the narrative flow to keep this theme central to the story, ensuring that each section contributes meaningfully to the theme.
  4. The revised outline should be detailed in plain text, with each bullet point clearly articulating the specific aspect of the data story it addresses.
  5. Use sub-bullet points to elaborate on complex points or to incorporate multiple data insights.

  ### Additional Guidelines:
  - The output must be in plain text and structured in bullet points.
  - Generate the response outline in between two <outline> tags.
  '''
  # ==================================================================================================================================

  return system_outline_revision_prompt, user_outline_revision_prompt


### **Narration**

In [ ]:
def initialize_narration_prompt(intention):
  # NARRATION
  # ======================================== $$Initial Narration with Intent$$ ======================================================
  initial_system_narration_prompt = f'You are an expert at generating engaging data stories. The generated data story should cover every important detail that can be observed in the data tables. Pay attention to small details and nuances as well as any trends or outliers in the given tables.'
  initial_user_narration_prompt = f'''### Task Description:
  Given a outline and the data tables corresponding to a data story in the input, you have the following tasks:
  1. Follow the outline rigorously to create a "data story" that will be highly informative and engaging to the audience.
  2. Highlight key statistics that are critical to understanding the theme. Explain these elements in a way that balances technical accuracy with accessibility, ensuring that your narrative is approachable for a non-specialist audience while still offering depth for those more familiar with the subject matter. Think about the narrative flow and how each piece of data contributes to the overall story arc.
  3. The overarching theme, denoted as "{intention}", should be the narrative’s backbone. Ensure that this theme resonates throughout the story, tying together different data points and insights into a coherent whole.
  4. In the outline, if it is mentioned to include a visualization, then include a ‘visualization placeholder’. Each visualization placeholder should also suggest a narrative element that the visualization supports or explains.
  5. Ensure that each portion of text in the story is in between two <text> tags.
  6. The visualization placeholder must contain necessary information about the visualization, such as:
   - chart title
   - chart type
   - x-axis and y-axis labels
   - x-axis data values and y-axis data values, etc.
  7. The visualizations must be put in between two <visualization> tags.
  8. Finally, make sure that the story is engaging to the audience.

  ### Additional Guidelines:
  - The output must be in plain text.
  - Generate the response narration in between two <narration> tags.
  '''

  return initial_system_narration_prompt, initial_user_narration_prompt


def narration_verification_prompt(intention):
  # ======================================== $$Narration Verification with Intent$$ =====================================================
  system_narration_verification_prompt = f'You are an intelligent critic, whose job is to identify inconsistencies between data presented in data tables, and an outline and a data story. Pay attention to small details and nuances as well as any trends or outliers in the given tables.'
  user_narration_verification_prompt = f'''### Task Description:
  Given the outline, the data tables and a data story in the input, you have the following tasks:
  1. Examine the data presented in the tables, the story’s outline, and the narrative itself. Look for discrepancies, inaccuracies, or any details that do not align.
  2. Provide a step-by-step analysis, highlighting specific data points and narrative elements that contribute to these inconsistencies.
  3. Make sure the story fully aligns with the intention or the main theme: "{intention}"
  4. Based on your analysis, draft a revision plan to refine the data story. Your plan should address identified inconsistencies and enhance theme alignment. Otherwise output: ‘No revision needed’.
  5. The output must be coherent, logically structured, and detailed, aiming for constructive feedback that enhances the data story’s impact.

  ### Additional Guidelines:
  - The output must be in plain text and in bullet points.
  - Generate the response narration in between two <narration> tags.
  '''

  return system_narration_verification_prompt, user_narration_verification_prompt

def narration_revision_prompt(intention):
  # ======================================== $$Narration Revision with Intent$$ =========================================================
  system_narration_revision_prompt = f'You are an expert at generating engaging data stories. The generated data story will cover every important detail that can be observed in the data tables. Pay attention to small details and nuances as well as any trends or outliers in the given tables.'
  user_narration_revision_prompt = f'''### Task Description:
  Given the data tables, the outline, the revision plan, and the data story in the input, your task is the following:
  1. Revise the data story according to the revision plan. Use the provided outline as your guide, adjusting the narrative according to the revision plan.
  2. The overarching theme, denoted as "{intention}", should be the narrative’s backbone.
  3. Ensure that this theme resonates throughout the story, tying together different data points and insights into a coherent whole.
  4. In the outline, if it is mentioned to include a visualization, then include a ‘visualization placeholder’. Each visualization placeholder should also suggest a narrative element that the visualization supports or explains.
  5. Ensure that each portion of text in the story is in between two <text> tags.
  6. The visualization placeholder must contain necessary information about the visualization, such as:
   - chart title
   - chart type
   - x-axis and y-axis labels
   - x-axis data values and y-axis data values, etc.
  7. The visualizations must be put in between two <visualization> tags.
  8. Finally, make sure that the story is engaging to the audience.

  ### Additional Guidelines:
  - The output must be in plain text.
  - Generate the response narration in between two <narration> tags.
  '''
  # =====================================================================================================================================

  return system_narration_revision_prompt, user_narration_revision_prompt


### **Multi-Agent LLM-Framework (GPT-Agent)**

In [ ]:
!pip install scipy --quiet
!pip install tenacity --quiet
!pip install tiktoken --quiet
!pip install termcolor --quiet
!pip install openai --quiet

In [ ]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API Key: "
)

openai_model = "NAME_OF_GPT_MODEL"

In [ ]:
import json
import openai
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-4o"
client = OpenAI(api_key=openai_apikey)

In [ ]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    # print(model)
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
            max_tokens=4096,
        )
        return response
    # except Exception as e:
    #     print("Unable to generate ChatCompletion response")
    #     print(f"Exception: {e}")
    except openai.APIError as e:
        #Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")
    except openai.APIConnectionError as e:
        #Handle connection error here
        print(f"Failed to connect to OpenAI API: {e}")
    except openai.RateLimitError as e:
        #Handle rate limit error (we recommend using exponential backoff)
        print(f"OpenAI API request exceeded rate limit: {e}")
        return e

## **ACTOR - CRITIC Agent RUN Start!!!**

In [ ]:
test_df['reflection'] = ''
test_df['reflection_revision_plan'] = ''
test_df['revised_reflection'] = ''
test_df['outline_int'] = ''
test_df['outline_int_revision_plan'] = ''
test_df['revised_outline_int'] = ''
test_df['outline_noint'] = ''
test_df['outline_noint_revision_plan'] = ''
test_df['revised_outline_noint'] = ''
test_df['narration_int'] = ''
test_df['narration_int_revision_plan'] = ''
test_df['revised_narration_int'] = ''
test_df['narration_noint'] = ''
test_df['narration_noint_revision_plan'] = ''
test_df['revised_narration_noint'] = ''

In [ ]:
start = time.time()
for i in range(len(test_df)):
  intention = test_df.article_title.iloc[i]
  res = test_df.paragraph_table_pair.iloc[i]
  tables = ''
  for x, item in enumerate(res):
    tables += f'### Table_{x}:\n{item["table"]}\n'

  ## ---------------------------------------------------------------------------------------------------------------------
  ## Step - 1: First Stage -- REFLECTION:::

  initial_system_reflection_prompt, initial_user_reflection_prompt = initialize_reflection_prompt()
  system_reflection_verification_prompt, user_reflection_verification_prompt = reflection_verification_prompt()
  system_reflection_revision_prompt, user_reflection_revision_prompt = reflection_revision_prompt()

  ## GPT-4o call for generate initial reflection
  final_initial_reflection_prompt = f'''{initial_user_reflection_prompt}\n### INPUT:\n### Tables:\n{tables}\n### OUTPUT:'''

  # print(f'Initial system reflection prompt:\n{initial_system_reflection_prompt}\n\nInitial user reflection prompt:\n{initial_user_reflection_prompt}')
  # print(f'Final prompt for initial reflection:\n{final_initial_reflection_prompt}')
  messages = generate_input_message(initial_system_reflection_prompt, final_initial_reflection_prompt)
  chat_response = chat_completion_request(
      messages, model=GPT_MODEL
  )

  try:
    reflection = chat_response.choices[0].message.content
    test_df['reflection'].iloc[i] = reflection
  except:
    reflection = 'Error'
    test_df['reflection'].iloc[i] = reflection
    continue

  # print(f'Reflection:\n{reflection}')
  # print('-'*60, '\n\n')
  ## ---------------------------------------------------------------------------------------------------------------------

  ## ---------------------------------------------------------------------------------------------------------------------
  ## GPT-4o call for generate reflection verification
  final_reflection_verification_prompt = f'''{user_reflection_verification_prompt}\n### INPUT:\n### Tables:\n{tables}\n### Reflection:\n{reflection}\n### OUTPUT:'''

  # print(f'Initial system reflection verification prompt:\n{system_reflection_verification_prompt}\n\nInitial user reflection verification prompt:\n{user_reflection_verification_prompt}')
  # print(f'Final prompt for initial reflection verification: \n{final_reflection_verification_prompt}')
  messages = generate_input_message(system_reflection_verification_prompt, final_reflection_verification_prompt)
  chat_response = chat_completion_request(
      messages, model=GPT_MODEL
  )

  try:
    reflection_revision_plan = chat_response.choices[0].message.content
    test_df['reflection_revision_plan'].iloc[i] = reflection_revision_plan
  except:
    reflection_revision_plan = 'Error'
    test_df['reflection_revision_plan'].iloc[i] = reflection_revision_plan
    continue

  # print(f'Reflection revision plan:\n{reflection_revision_plan}')
  # print('-'*60, '\n\n')
  ## ---------------------------------------------------------------------------------------------------------------------

  ## ---------------------------------------------------------------------------------------------------------------------
  ## GPT-4o call for generate revised reflection
  final_reflection_revision_prompt = f'''{user_reflection_revision_prompt}\n### INPUT:\n### Tables:\n{tables}\n### Previous Reflection:\n{reflection}\n### Revision Plan:\n{reflection_revision_plan}\n### OUTPUT:'''

  # print(f'Initial system reflection revision prompt:\n{system_reflection_revision_prompt}\n\nInitial user reflection revision prompt:\n{user_reflection_revision_prompt}')
  # print(f'Final prompt for initial reflection revision:\n{final_reflection_revision_prompt}')

  messages = generate_input_message(system_reflection_revision_prompt, final_reflection_revision_prompt)
  chat_response = chat_completion_request(
      messages, model=GPT_MODEL
  )

  try:
    final_reflection = chat_response.choices[0].message.content
    test_df['revised_reflection'].iloc[i] = final_reflection
  except:
    reflection_revision_plan = 'Error'
    test_df['revised_reflection'].iloc[i] = final_reflection
    continue

  # print(f'Final Reflection:\n{final_reflection}')
  # print('-'*60, '\n\n')
  ## xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


  ## Step - 2.1: Second Stage -- OUTLINE GENERATION:::

  initial_system_outline_prompt_int, initial_user_outline_prompt_int = initialize_outline_prompt(intention)
  system_outline_verification_prompt_int, user_outline_verification_prompt_int =  outline_verification_prompt(intention)
  system_outline_revision_prompt_int, user_outline_revision_prompt_int = outline_revision_prompt(intention)


  ## ---------------------------------------------------------------------------------------------------------------------
  ## GPT-4o call for generate initial outline
  final_outint_prompt = f'''{initial_user_outline_prompt_int}\n### INPUT:\n### Tables:\n{tables}\n### Reflection:\n{reflection}\n### OUTPUT:'''

  # print(f'Initial system outline prompt:\n{initial_system_outline_prompt_int}\n\nInitial user outline prompt:\n{initial_user_outline_prompt_int}')
  # print(f'Final prompt for initial outline with intention:\n{final_outint_prompt}')

  messages = generate_input_message(initial_system_outline_prompt_int, final_outint_prompt)
  chat_response = chat_completion_request(
      messages, model=GPT_MODEL
  )

  try:
    outline_int = chat_response.choices[0].message.content
    test_df['outline_int'].iloc[i] = outline_int
  except:
    outline_int = 'Error'
    test_df['outline_int'].iloc[i] = outline_int
    continue

  # print(f'Outline with intention:\n{outline_int}')
  # print('-'*60, '\n\n')

  ## ---------------------------------------------------------------------------------------------------------------------

  ## ---------------------------------------------------------------------------------------------------------------------
  ## GPT-4o call for generate outline revision plan
  final_outint_ver_prompt = f'''{user_outline_verification_prompt_int}\n### INPUT:\n### Tables:\n{tables}\n### Reflection:\n{reflection}\n### Outline:\n{outline_int}\n### OUTPUT:'''

  # print(f'Initial system outline prompt:\n{system_outline_verification_prompt_int}\n\nInitial user outline prompt:\n{initial_user_outline_prompt_int}')
  # print(f'Final prompt for initial outline with intention:\n{final_outint_ver_prompt}')

  messages = generate_input_message(system_outline_verification_prompt_int, final_outint_ver_prompt)
  chat_response = chat_completion_request(
      messages, model=GPT_MODEL
  )

  try:
    outline_int_revision_plan = chat_response.choices[0].message.content
    test_df['outline_int_revision_plan'].iloc[i] = outline_int_revision_plan
  except:
    outline_int_revision_plan = 'Error'
    test_df['outline_int_revision_plan'].iloc[i] = outline_int_revision_plan
    continue

  # print(f'Outline with intention revision plan:\n{outline_int_revision_plan}')
  # print('-'*60, '\n\n')
  ## ---------------------------------------------------------------------------------------------------------------------

  ## ---------------------------------------------------------------------------------------------------------------------
  ## GPT-4o call for generate revised outline
  final_outint_rev_prompt = f'''{user_outline_revision_prompt_int}\n### INPUT:\n### Tables:\n{tables}\n### Previous Outline:\n{outline_int}\n### Revision Plan:\n{outline_int_revision_plan}\n### OUTPUT:'''

  # print(f'Initial system outline revision plan prompt:\n{system_outline_revision_prompt_int}\n\nInitial user outline revision plan prompt:\n{initial_user_outline_prompt_int}')
  # print(f'Final prompt for outline revision plan with intention: \n{final_outint_rev_prompt}')

  messages = generate_input_message(system_outline_revision_prompt_int, final_outint_rev_prompt)
  chat_response = chat_completion_request(
      messages, model=GPT_MODEL
  )

  try:
    final_outline_int = chat_response.choices[0].message.content
    test_df['revised_outline_int'].iloc[i] = final_outline_int
  except:
    final_outline_int = 'Error'
    test_df['revised_outline_int'].iloc[i] = final_outline_int
    continue

  # print(f'Outline with intention revision:\n{final_outline_int}')
  # print('-'*60, '\n\n')
  ## ---------------------------------------------------------------------------------------------------------------------
  ## xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx


  ## Step - 3.1: Third Stage -- NARRATION GENERATION:::

  initial_system_narration_prompt_int, initial_user_narration_prompt_int = initialize_narration_prompt(intention)
  system_narration_verification_prompt_int, user_narration_verification_prompt_int = narration_verification_prompt(intention)
  system_narration_revision_prompt_int, user_narration_revision_prompt_int = narration_revision_prompt(intention)

  ## ---------------------------------------------------------------------------------------------------------------------
  ## GPT-4o call for generate initial narration
  final_narrint_prompt = f'''{initial_user_narration_prompt_int}\n### INPUT:\n### Tables:\n{tables}\n### Outline:\n{final_outline_int}\n### OUTPUT:'''

  # print(f'Initial system narration int prompt:\n{initial_system_narration_prompt_int}\n\nInitial user narration int prompt:\n{initial_user_narration_prompt_int}')
  # print(f'Final prompt for initial narration with intention:\n{final_narrint_prompt}')

  messages = generate_input_message(initial_system_narration_prompt_int, final_narrint_prompt)
  chat_response = chat_completion_request(
      messages, model=GPT_MODEL
  )

  try:
    narration_int = chat_response.choices[0].message.content
    test_df['narration_int'].iloc[i] = narration_int
  except:
    narration_int = 'Error'
    test_df['narration_int'].iloc[i] = narration_int
    continue

  # print(f'Narration with intention:\n{narration_int}')
  # print('-'*60, '\n\n')
  ## ---------------------------------------------------------------------------------------------------------------------

  ## ---------------------------------------------------------------------------------------------------------------------
  ## GPT-4o call for generate narration revision plan
  final_narrint_ver_prompt = f'''{user_narration_verification_prompt_int}\n### INPUT:\n### Tables:\n{tables}\n### Outline:\n{final_outline_int}\n### Data Story:\n{narration_int}\n### OUTPUT:'''

  # print(f'Initial system narration verification prompt:\n{system_narration_verification_prompt_int}\n\nInitial user narration verification prompt:\n{user_narration_verification_prompt_int}')
  # print(f'Final prompt for narration verification prompt with intention:\n{final_narrint_ver_prompt}')

  messages = generate_input_message(system_narration_verification_prompt_int, final_narrint_ver_prompt)
  chat_response = chat_completion_request(
      messages, model=GPT_MODEL
  )

  try:
    narration_int_revision_plan = chat_response.choices[0].message.content
    test_df['narration_int_revision_plan'].iloc[i] = narration_int_revision_plan
  except:
    narration_int_revision_plan = 'Error'
    test_df['narration_int_revision_plan'].iloc[i] = narration_int_revision_plan
    continue

  # print(f'Narration with intention revision plan:\n{narration_int_revision_plan}')
  # print('-'*60, '\n\n')
  ## ---------------------------------------------------------------------------------------------------------------------

  ## ---------------------------------------------------------------------------------------------------------------------
  ## GPT-4o call for generate narration revision plan
  final_narrint_rev_prompt = f'''{user_narration_revision_prompt_int}\n### INPUT:\n### Tables:\n{tables}\n### Outline:\n{final_outline_int}\n### Previous Data Story:\n{narration_int}\n### Revision Plan:\n{narration_int_revision_plan}\n### OUTPUT:'''

  # print(f'Initial system narration revision prompt:\n{system_narration_revision_prompt_int}\n\nInitial user outline prompt:\n{user_narration_revision_prompt_int}')
  # print(f'Final prompt for narration revision prompt with intention:\n{final_narrint_rev_prompt}')

  messages = generate_input_message(system_narration_revision_prompt_int, final_narrint_rev_prompt)
  chat_response = chat_completion_request(
      messages, model=GPT_MODEL
  )

  try:
    final_narration_int = chat_response.choices[0].message.content
    test_df['revised_narration_int'].iloc[i] = final_narration_int
  except:
    final_narration_int = 'Error'
    test_df['revised_narration_int'].iloc[i] = final_narration_int
    continue

  # print(f'Narration with intention revised:\n{final_narration_int}')
  # print('-'*60, '\n\n')
  ## ---------------------------------------------------------------------------------------------------------------------

  ## xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

  if i%10 == 0 and i!=0:
    test_df.to_json(os.path.join(data, f'/path/to/save_dir/stories.json'), orient='records', lines=True)
    print(f'Iteration {i} complete, time taken: {time.time()-start}s')
    print('-'*50)
    start = time.time()
